In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Imports

In [ ]:
from typing import Optional

from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langsmith import traceable
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

load_dotenv()

# Vanilla workflow

In [ ]:
class Evaluation(BaseModel):
    explanation: str = Field(
        description="Explain why the text evaluated matches or not the evaluation criteria"
    )
    feedback: str = Field(
        description="Provide feedback to the writer to improve the text"
    )
    is_correct: bool = Field(
        description="Whether the text evaluated matches or not the evaluation criteria"
    )


class State(BaseModel):
    topic: str
    article: Optional[str] = None
    evaluation: Optional[Evaluation] = None


model = ChatOpenAI(model="gpt-4.1-mini")


@traceable
def evaluate_text(state: State) -> Evaluation:
    model_with_str_output = model.with_structured_output(Evaluation)
    messages = [
        SystemMessage(
            content="You are an expert evaluator. Provided with a text, you will evaluate if it's written in British English and if it's appropriate for a young audience. The text must always use British spelling and grammar. Make sure the text doesn't include any em dash."
        ),
        HumanMessage(content=f"Evaluate the following text:\n\n{state.output}"),
    ]
    response = model_with_str_output.invoke(messages)
    return response


@traceable
def fix_text(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer. Provided with a text, you will fix the text to improve it."
        ),
        HumanMessage(
            content=f"You were tasked with writing an article about {state.topic}. You wrote the following text:\n\n{state.article}\n\nYou've got the following feedback:\n\n{state.evaluation.feedback}\n\nFix the text to improve it."
        ),
    ]
    response = model.invoke(messages)
    return response.content


@traceable
def generate_text(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer. Provided with a topic, you will generate an engaging article with less than 500 words."
        ),
        HumanMessage(content=f"Generate a text about this topic:\n\n{state.topic}"),
    ]
    response = model.invoke(messages)
    return response.content


@traceable
def generate_text_dispatch(state: State) -> str:
    if state.evaluation:
        return fix_text(state)
    return generate_text(state)


@traceable
def run_pipeline(topic: str) -> State:
    state = State(topic=topic)

    for _ in range(4):
        state.article = generate_text_dispatch(state)
        state.evaluation = evaluate_text(state)
        if state.evaluation.is_correct:
            return state

    return state


state = run_pipeline("Substance abuse of athletes")

# LangGraph implementation

## Exercise:

Implement the same workflow with LangGraph